In [ ]:
!pip install contractions
!pip install TextBlob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Sun May  8 19:33:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+------

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from wordcloud import WordCloud
import contractions
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import re

In [ ]:
"""
Getting information about dataset
"""
cyble_data=pd.read_csv('/content/drive/MyDrive/NLP_dataset/dataset_2_2.csv')


In [ ]:
cyble_data.head()

,Unnamed: 0,Location,Text,Sentiment,Likes,quotes,replies,retweet
0,0,NewYork,Woke criminal New York Times brings back Covid...,Negative,1.0,0.0,1.0,0.0
1,1,NewYork,He proven New York liberal narcissist hired wo...,Negative,1.0,0.0,0.0,0.0
2,2,NewYork,CBS2 News At 11 New York leader say mask vacci...,Neutral,5.0,0.0,0.0,3.0
3,3,NewYork,Pediatric COVID Deaths New York,Neutral,3.0,0.0,0.0,1.0
4,4,NewYork,The 7 day rolling average new Covid hospitaliz...,Negative,2.0,0.0,0.0,0.0


In [ ]:
cyble_data=cyble_data[['Location', 'Text', 'Sentiment']]
cyble_data.head()

,Location,Text,Sentiment
0,NewYork,Woke criminal New York Times brings back Covid...,Negative
1,NewYork,He proven New York liberal narcissist hired wo...,Negative
2,NewYork,CBS2 News At 11 New York leader say mask vacci...,Neutral
3,NewYork,Pediatric COVID Deaths New York,Neutral
4,NewYork,The 7 day rolling average new Covid hospitaliz...,Negative


In [ ]:
cyble_data = cyble_data.rename(columns={'Sentiment': 'Section'})
cyble_data.head()

,Location,Text,Section
0,NewYork,Woke criminal New York Times brings back Covid...,Negative
1,NewYork,He proven New York liberal narcissist hired wo...,Negative
2,NewYork,CBS2 News At 11 New York leader say mask vacci...,Neutral
3,NewYork,Pediatric COVID Deaths New York,Neutral
4,NewYork,The 7 day rolling average new Covid hospitaliz...,Negative


In [ ]:
cyble_data['Location'].value_counts()

Singapore       47340
London          47194
Mumbai          46537
NewYork         46489
Paris           45634
Sydney          44527
Berlin          44465
Tokyo           42635
Johannesburg    24782
Name: Location, dtype: int64

In [ ]:
cyble_data=cyble_data.sample(frac=1)
cyble_data.head()

,Location,Text,Section
23395,London,It always obvious got covid A bit tired achy A...,Negative
320734,Tokyo,Pfizer expects 54 billion 2022 sale Covid vacc...,Neutral
20968,London,Leonard Barden chess Carlsen overcomes Covid R...,Neutral
22522,London,Did wear one Covid stop flu cold transmission ...,Negative
329211,Johannesburg,damn girl catalytic converter want steal,Negative


In [ ]:
"""
Dataset Visual analysis
"""
sentiment_count=cyble_data['Section'].value_counts()
plt.pie(sentiment_count, labels=sentiment_count.index,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.show()

sentiment_list=dict(cyble_data['Section'].value_counts())

"""
Building Wordcloud

for key in sentiment_list:
    distinct_sentiment=cyble_data[cyble_data['Section']==key]
    text1= " ".join(text.lower() for text in distinct_sentiment['Text'])
    wordcloud=WordCloud().generate(text1)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

"""

In [ ]:
#text contraction
"""
text contraction means that changing don't to donot etc.'
"""
cyble_data=cyble_data.apply(lambda x: x.astype(str).str.lower())

cyble_data=cyble_data.reset_index(drop=True)


cyble_data['Text']=cyble_data['Text'].apply(contractions.fix)    

"""
Text Tokenization
"""
from nltk import word_tokenize
from textblob import TextBlob
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words=set(stopwords.words('english'))



def tokenize_text(text,
                  keep_punctuation=False,
                   keep_alphanum=False,
                  keep_stop=False
                  ):
    
    tokenized_list=TextBlob(text)
    tokenized_list=tokenized_list.words
    
    if not keep_punctuation:
        tokenized_list=[token for token in tokenized_list 
                        if token not in string.punctuation]
    
    if not keep_alphanum:
        tokenized_list=[token for token in tokenized_list if token.isalpha()]
        
    if not keep_stop:
        # stop_words=set(stop_words.words('english'))
        tokenized_list=[token for token in tokenized_list if not token in stop_words]
        
    return tokenized_list

# temp_text='Krishna yadav were 7 '
# temp_text=tokenize_text(temp_text)

cyble_data['Text']=cyble_data['Text'].apply(tokenize_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
cyble_data['Text'].head()

0    [always, obvious, got, covid, bit, tired, achy...
1    [pfizer, expects, billion, sale, covid, vaccin...
2    [leonard, barden, chess, carlsen, overcomes, c...
3    [wear, one, covid, stop, flu, cold, transmissi...
4      [damn, girl, catalytic, converter, want, steal]
Name: Text, dtype: object

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')

# import nltk
# dler = nltk.downloader.Downloader()
# dler._update_index()
# dler.download('omw-1.4')

def lemmatize_text(text):
    tokenized_list_lematized=[]
    lemmatizer=WordNetLemmatizer()
    for token in text:
        tokenized_list_lematized.append(lemmatizer.lemmatize(token))
    
    return tokenized_list_lematized

# temp_text=['Manager','Managing', 'Management']
# returned_text=lemmatize_text(temp_text)

cyble_data['Text']=cyble_data['Text'].apply(lemmatize_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Singapore       47340
# London          47194
# Mumbai          46537
# NewYork         46489
# Paris           45634
# Sydney          44527
# Berlin          44465
# Tokyo           42635
# Johannesburg    24782

locations=cyble_data['Location'].unique()
data_list=[]

for location in locations:
  temp=pd.DataFrame(
      
      
  )


In [ ]:
print(locations)

['london' 'tokyo' 'johannesburg' 'paris' 'berlin' 'singapore' 'sydney'
 'newyork' 'mumbai']


In [ ]:
"""
Text Vectorization
"""

from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
cyble_data['category']=encoder.fit_transform(cyble_data['Section'])

In [ ]:
cyble_data['Text'].head()

0    [always, obvious, got, covid, bit, tired, achy...
1    [pfizer, expects, billion, sale, covid, vaccin...
2    [leonard, barden, chess, carlsen, overcomes, c...
3    [wear, one, covid, stop, flu, cold, transmissi...
4      [damn, girl, catalytic, converter, want, steal]
Name: Text, dtype: object

In [ ]:
#detokenize the word
from nltk.tokenize.treebank import TreebankWordDetokenizer

def detokenize(text):
    detokenized=TreebankWordDetokenizer().detokenize(text)
    return detokenized

cyble_data['Text']=cyble_data['Text'].apply(detokenize)

In [ ]:
cyble_data['Text'].head()

0    always obvious got covid bit tired achy headac...
1    pfizer expects billion sale covid vaccine trea...
2    leonard barden chess carlsen overcomes covid r...
3    wear one covid stop flu cold transmission kill...
4             damn girl catalytic converter want steal
Name: Text, dtype: object

In [ ]:
"""
creating n_grams
"""
import re

def generate_ngrams(s):
    n=1
    # Convert to lowercases
    s = s.lower()
    
    # # Replace all none alphanumeric characters with spaces
    # s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]


cyble_data['Text']=cyble_data['Text'].apply(generate_ngrams)


x=cyble_data['Text'].tolist()
y=cyble_data['category'].tolist()   

In [ ]:
#counting the lenth of a words
count=[]
for index in range(len(x)):
    count.append(len(x[index]))

print(np.average(count))

14.495545465512329


In [ ]:
#removing empty list        
x_revised=[]
y_revised=[]
empty=[]
for index in range(len(x)):
    if len(x[index])!=0:
        x_revised.append(x[index])
        y_revised.append(y[index])

In [ ]:
#onehot representation
voc_size=30000
flattened = []
for sublist in x:
    flattened.append(str(sublist)[1:-1])


onehot_representation=[one_hot(words, voc_size) for words in flattened]
# onehot_representation

In [ ]:
#embedding representaion
sentence_length=20
embedded_result=pad_sequences(onehot_representation, padding='pre', maxlen=sentence_length)

In [ ]:
embedded_result.shape

(389603, 20)

In [ ]:
import numpy as np
x_data=np.array(embedded_result)
y_data=np.array(y)

#train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
pd.DataFrame(x_train).shape

(311682, 20)

In [ ]:
#second model
feature_vector_embedding=20
model = keras.models.Sequential([
        keras.layers.Embedding(voc_size,feature_vector_embedding, input_length=sentence_length),
        # keras.layers.Dropout(0.5),
        keras.layers.GRU(30, return_sequences=True),
        keras.layers.Dropout(0.5),
        keras.layers.GRU(20, return_sequences=True),
        keras.layers.Dropout(0.2),
        keras.layers.GRU(10, return_sequences=True),
        keras.layers.GRU(5),
        keras.layers.Dense(3, activation="softmax")
    ])
model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])
history=model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=256)

In [ ]:
model.summary()